In [1]:
import numpy as np
import pandas as pd
import cv2
import os
import tqdm
from scipy.io import loadmat
import matplotlib.pyplot as plt
import glob
import pickle

In [6]:
import keras

Using TensorFlow backend.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\installation\envs\trevaller_policy\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-88d96843a926>", line 1, in <module>
    import keras
  File "C:\installation\envs\trevaller_policy\lib\site-packages\keras\__init__.py", line 3, in <module>
    from . import utils
  File "C:\installation\envs\trevaller_policy\lib\site-packages\keras\utils\__init__.py", line 6, in <module>
    from . import conv_utils
  File "C:\installation\envs\trevaller_policy\lib\site-packages\keras\utils\conv_utils.py", line 9, in <module>
    from .. import backend as K
  File "C:\installation\envs\trevaller_policy\lib\site-packages\keras\backend\__init__.py", line 1, in <module>
    from .load_backend import epsilon
  File "C:\installation\envs\trevaller_policy\lib\site-packages\keras\backend\load_backend.py", line 90, in <module>
    from .tensorflow_backend impo

ImportError: cannot import name 'context' from 'tensorflow.python.eager' (C:\Users\Sanjeev\AppData\Roaming\Python\Python37\site-packages\tensorflow_core\python\eager\__init__.py)

In [2]:
from keras import backend as K
import keras
import cv2
from Utils import *
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.layers import *
# from keras.applications.mobilenetv2 import MobileNetV2
from keras.applications.mobilenet_v2 import MobileNetV2
import tensorflow as tf
import numpy as np
from keras.models import Model
from keras.models import model_from_json
import matplotlib.pyplot as plt
import os
# from keras.layers import Input

Using TensorFlow backend.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Sanjeev\AppData\Roaming\Python\Python37\site-packages\tensorflow_core\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Users\Sanjeev\AppData\Roaming\Python\Python37\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\Users\Sanjeev\AppData\Roaming\Python\Python37\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\installation\envs\trevaller_policy\lib\imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "C:\installation\envs\trevaller_policy\lib\imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: The specified module could not be found.

During han

TypeError: can only concatenate str (not "list") to str

In [ ]:
def display_image(img,title='Image'):
    
    channel = len(img.shape)
    
    if channel <= 2:
        plt.imshow(img, cmap='gray')
        plt.title(title)
        plt.axis('off')
    else:
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        plt.imshow(img)
        plt.title(title)
        plt.axis('off')

In [ ]:
def display_bounding_box(img, word_bbox_list,title="word bounding box"):
    for bbox in word_bbox_list:
        b = bbox[-1]
        img = cv2.rectangle(img, (int(b[0]), int(b[1])), (int(b[2]), int(b[3])), (0, 255, 0), 2)

    # show annotated image and wait for keypress
    cv2.namedWindow(title, cv2.WINDOW_NORMAL)
    cv2.imshow(title, img)

In [ ]:
def get_bbox_from_text(text):
    bbox=[]
    
    for line in text.split('\n')[0:-1]:
        box = line.split(' ')[1:]
        bbox.append([float(i) for i in box])
    
    return tuple(bbox)

def from_yolo_to_bbox(img,yolo_bbox):
    bbox=[]
    img_h, img_w, _ = img.shape
    for box in yolo_bbox:
        x1, y1 = int((box[0] + box[2]/2)*img_w), int((box[1] + box[3]/2)*img_h)
        x2, y2 = int((box[0] - box[2]/2)*img_w), int((box[1] - box[3]/2)*img_h)
        bbox.append([x1, y1, x2, y2])
    return tuple(bbox)
    
def draw_boxes(imgs, boxes,title='bounding box'):
    for box in boxes:
        x1, y1, x2, y2 = box
        cv2.rectangle(img, (x1, y1), (x2, y2), (0,255,0), 3)
            
    cv2.namedWindow(title, cv2.WINDOW_NORMAL)
    cv2.imshow(title, img)

In [ ]:
# path declearation 
img_path ="C:/Users/Sanjeev/Downloads/software/0325updated.task2train(626p)-20200414T125907Z-001/data/temp/*.jpg"
bbox_path = "C:/Users/Sanjeev/Downloads/software/0325updated.task2train(626p)-20200414T125907Z-001/data/temp/*.txt"
save_path = "C:/Users/Sanjeev/Documents/Python Scripts/NLP/data_for_Save/"
# save_text= "C:/Users/Sanjeev/Downloads/software/0325updated.task2train(626p)-20200414T125907Z-001/data/yolo/text/"
# save_img="C:/Users/Sanjeev/Downloads/software/0325updated.task2train(626p)-20200414T125907Z-001/data/yolo/image/"
save_text= "C:/Users/Sanjeev/Downloads/software/0325updated.task2train(626p)-20200414T125907Z-001/data/yolodata/text/"
save_img="C:/Users/Sanjeev/Downloads/software/0325updated.task2train(626p)-20200414T125907Z-001/data/yolodata/image/"

In [ ]:
img_path_list = glob.glob(img_path)
bbox_path_list = glob.glob(bbox_path)
print(len(img_path_list),len(bbox_path_list))

In [ ]:
idx=1
with open(bbox_path_list[idx],'r') as f:
    text = f.read()
yolo_bbox =get_bbox_from_text(text)
yolo_bbox

In [ ]:

img = cv2.imread(img_path_list[idx])

In [ ]:
img = cv2.imread(img_path_list[idx])
img= cv2.resize(img,(512,512))
bbox = from_yolo_to_bbox(img,yolo_bbox)
bbox

In [ ]:
img = cv2.imread(img_path_list[idx])
img= cv2.resize(img,(512,512))
draw_boxes(img,bbox)
cv2.waitKey(0)

In [ ]:
import shutil

In [ ]:
# making label uniform 
for path in bbox_path_list:
#     print(path)
    with open(path, 'r') as file:
        text = file.read()
#         print(text)
    label = text.split("\n")[0]
    file_name= path.split("\\")[-1]
    file_name=file_name.split('.')[0]
    img_path = path.split("\\")[0]
    img_path =img_path +'/'+file_name+'.jpg'
#     w,h,_= cv2.imread(img_path).shape
#     print(img_path,'\n', save_img+file_name+'.jpg')
    shutil.copy(img_path, save_img+file_name+'.jpg')
   
    if label.split(' ')[0]=='0':
        with open(save_text+file_name+'.txt', 'w') as f:
            text=text.split("\n")[0:-1]
            for info in text:
                info=info.split(' ')
                info[0]='15'
                info=' '.join(info)
                f.write(info+'\n')
                print(info)
    
    

In [ ]:
# making label uniform 
for path in bbox_path_list:
#     print(path)
    with open(path, 'r') as file:
        text = file.read()
#         print(text)
    label = text.split("\n")[0]
    file_name= path.split("\\")[-1]
    file_name=file_name.split('.')[0]
    img_path = path.split("\\")[0]
    img_path =img_path +'/'+file_name+'.jpg'
    w,h,_= cv2.imread(img_path).shape
    print(w,h,_)
    print(img_path)
#     print(img_path,'\n', save_img+file_name+'.jpg')
#     shutil.copy(img_path, save_img+file_name+'.jpg')
   
    if label.split(' ')[0]=='0':
        with open(save_text+file_name+'.txt', 'w') as f:
            text=text.split("\n")[0:-1]
            for info in text:
                info=info.split(' ')
#                 print(float(info[1]))
#                 print(type(float(info[1])))
                info = [float(i) for i in info]
                info[0]='15'
#                 info=' '.join(info)
#                 f.write(info+'\n')
#                 print(info)
                print(int(info[1]*2*h), int(info[2]*2*w), int(info[3]*h),int(info[4]*w))
#                 print(info)
    
    

In [ ]:
bbox_file= "C:/Users/Sanjeev/Downloads/software/0325updated.task2train(626p)-20200414T125907Z-001/data/yolodata/text/*.txt"
img_file="C:/Users/Sanjeev/Downloads/software/0325updated.task2train(626p)-20200414T125907Z-001/data/yolodata/image/*.jpg"

In [ ]:
bbox_path_list[0]

In [ ]:
image_path = './git_project/Text-Detection-using-Yolo-Algorithm-in-keras-tensorflow/Data/images/'
gt_path = './git_project/Text-Detection-using-Yolo-Algorithm-in-keras-tensorflow/Data/ground_truth/'
train_image_paths = []
train_gt_paths = []


for new_file in tqdm.tqdm(os.listdir(gt_path)):
    
    name_split = new_file.split('.')
    image_name = name_split[0][3:]
    image_name = image_name + '.jpg'
    
    if 'gt' in new_file:
        image_name = name_split[0][3:]
        image_name = image_name + '.jpg'
    
    path_img = os.path.join(image_path , image_name)
    train_image_paths.append(path_img)
    train_gt_paths.append(os.path.join(gt_path , new_file))

In [ ]:
X_final = []
Y_final = []
grid_h = 16
grid_w = 16
img_w = 512
img_h = 512



for z in tqdm.tqdm(range(len(train_image_paths))):
    
    new_file = train_image_paths[z]
    print('new_file : ',new_file)
    #print(new_file)
    x = cv2.imread(train_image_paths[z])
    x_sl = 512/x.shape[1]
    y_sl = 512/x.shape[0]
    print('x_sl : ',x_sl,y_sl)
    img = cv2.resize(x,(512,512))
    
    
    X_final.append(img)
    
    #plt.imshow(cv2.imread(new_file))
    #plt.show()
    
    i = " "
    
    if 'img' in new_file:
        i = ", "
    
    Y = np.zeros((grid_h,grid_w,1,5))
    
    file = train_gt_paths[z]
    print('file : ',file)
    name = open(file , 'r')
    data = name.read()
    print('data : ',data)
    data = data.split("\n")
    data = data[:-1]
    print('data [-1]: ',data)
    
    for li in data:
        temp_list = []
        file_data = li.split(i)
        print('file_data: ',file_data)
        strr = file_data[4]
        print('strr: ',strr)
        bb = file_data[:4]
        print('bb: ',bb)
        xmin = int(bb[0])*x_sl
        xmax = int(bb[2])*x_sl
        ymin = int(bb[1])*y_sl
        ymax = int(bb[3])*y_sl
        print('ymax  : ',xmin,xmax,ymin,ymax)
        #te = cv2.rectangle(img,(int(xmin),int(ymin)),(int(xmax),int(ymax)) , color = (0,255,0))
        
        w = (xmax - xmin)/img_w
        h = (ymax - ymin)/img_h
        print('w,h  : ',w,h)
        x = ((xmax + xmin)/2)/img_w
        y = ((ymax + ymin)/2)/img_h
        print('x,y  : ',x,y)
        x = x * grid_w
        y = y * grid_h
        
        print('x,y  : ',x,y)
        
        Y[int(y),int(x),0,0] = 1
        Y[int(y),int(x),0,1] = x - int(x)
        Y[int(y),int(x),0,2] = y - int(y)
        Y[int(y),int(x),0,3] = w
        Y[int(y),int(x),0,4] = h
        
        print('y  : ',Y)
#         break
    #plt.imshow(te)
    #plt.show()
    print('final y  : ',Y)
    Y_final.append(Y)
    break




# X = np.array(X_final)
# X_final = []
# Y = np.array(Y_final)
# Y_final = []

# X = (X - 127.5)/127.5

# np.save('Data1/X.npy',X)
# np.save('Data1/Y.npy',Y)

In [ ]:
411*.8

In [ ]:
from keras.layers import Input

In [ ]:

# model layout
def yolo_model(input_shape):
    
    
    inp = Input(input_shape)
   
    model = MobileNetV2( input_tensor= inp , include_top=False, weights='imagenet')
    last_layer = model.output
    
    conv = Conv2D(512,(3,3) , activation='relu' , padding='same')(last_layer)
    conv = Dropout(0.4)(conv)
    bn = BatchNormalization()(conv)
    lr = LeakyReLU(alpha=0.1)(bn)
    
    
    conv = Conv2D(128,(3,3) , activation='relu' , padding='same')(lr)
    conv = Dropout(0.4)(conv)
    bn = BatchNormalization()(conv)
    lr = LeakyReLU(alpha=0.1)(bn)
    
    
    conv = Conv2D(5,(3,3) , activation='relu' , padding='same')(lr)
    
    final = Reshape((grid_h,grid_w,classes,info))(conv)
    
    model = Model(inp,final)
    
    return model
    


In [ ]:
#load and save model
img_h,img_w,channels = (28,28,3)

input_size = (img_h,img_w,channels)

model = yolo_model(input_size)

print(model.summary())


# final code from here 